- 查看指令微调的数据文件

In [2]:
import jsonlines

# 查看JSONL文件的前几行
file_path = './dataset/sft_data_zh.jsonl'

with jsonlines.open(file_path) as reader:
    for i, obj in enumerate(reader):
        print(obj)  # 打印每一行
        if i >= 4:  # 只打印前5行
            break

{'id': 1, 'instruction': '', 'input': '好的。现在请你将这个文本中的所有的逗号都替换成空格。', 'output': '好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？', 'history': [['给定一段文本和关键词列表，删除文本中包含所有给定关键词的子字符串。\n文本："这是一个测试句子，目的是看看模型是否可以正确地从这个句子中删除关键词。"\\n关键词列表：[‘测试’，‘模型’]', '删除包含所有给定关键词的子字符串后，文本变为："这是一个句子，目的是看看是否可以正确地从这个句子中删除关键词。"']], 'language': 'chinese', 'data_source': 'https://huggingface.co/datasets/BelleGroup/train_3.5M_CN', 'input_len': 59, 'output_len': 66, 'num_utter': 2, 'type': 31, 'type_keyword': ['字符串', '代码', '函数', '编写', '实现', '给定', '使用', '输入', '文本', '程序']}
{'id': 2, 'instruction': '', 'input': '帮我回答一道历史题目。清朝时期的八旗共有多少旗人？', 'output': '清朝时期八旗旗人总数约为200万人左右，其中正黄旗、正蓝旗、正白旗、正红旗的人数较多，其他旗的人数较少。', 'history': [['写一个关于秋天的诗吧。', '红黄绿交织，田野金黄色。风吹枫叶落，人在秋天慢步行。'], ['给我推荐一些好听的摇滚歌曲。', "摇滚歌曲推荐：AC/DC的Back in Black，Guns N' Roses的Sweet Child O' Mine，Rolling Stones的Satisfaction，Led Zeppelin的Stairway to Heaven，Nirvana的Smells Like Teen Spirit等。"], ['计算一下2的100次方。', '2的100次方等于1.26765060022822940149670320537

In [37]:
import pandas as pd

# 加载 CSV 文件
file_path = './dataset/sft_data_single.csv'  # 确保路径正确

# 使用 pandas 读取前 5 行
df = pd.read_csv(file_path, nrows=5)

# 显示前 5 行数据
print(df)

  history                                q  \
0      []           请从这篇文章中提取出关于分类垃圾的解决方案。   
1      []  根据以下文本，对此事件进行分类：中国队在足球比赛中赢得了冠军。   
2      []                     请告诉我什么是机器学习。   
3      []                    请解释一下什么是人工智能。   
4      []           对给定的命题进行真假判断\n梦是一种真实经验   

                                                   a  
0  文章中提到的第二个方面——分类垃圾，是解决垃圾污染的有效措施之一。垃圾分类是指将生活垃圾按照...  
1  这个事件可以被分类为体育比赛。具体地，中国队在足球比赛中致胜并赢得了冠军。由此可以推断出，这...  
2  机器学习是一种人工智能技术，它使用算法和数学模型，让计算机自动地从数据中学习，并根据这些数据...  
3  人工智能是一种模拟人类智能的技术和方法。它的发展包括机器学习、自然语言处理、计算机视觉等技术...  
4  这个命题是错误的。尽管梦境中的经验可以让我们产生强烈的情感反应，但梦并不是一种真实经验，它是...  


- 导入必要的库

In [17]:
import csv
import itertools
import re
import json
import jsonlines
import psutil
import ujson
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

In [18]:
bos_token = "<s>"
eos_token = "</s>"

In [19]:
tokenizer = AutoTokenizer.from_pretrained('./model/mateconv_tokenizer', use_fast=False)
print('tokenizer词表大小：', len(tokenizer))

tokenizer词表大小： 6400


- 修改数据集类型

In [15]:
from tqdm import tqdm  # 导入 tqdm

def sft_process(contain_history=False):
    file_name = 'sft_data.csv' if contain_history else 'sft_data_single.csv'

    def chinese_ratio(text):
        """计算文本中中文字符的比例。"""
        chinese_chars = re.findall(r'[\u4e00-\u9fff]', text)  # 匹配中文字符
        return len(chinese_chars) / len(text) if text else 0

    def process_and_write_data(data):
        """处理数据并将其写入 CSV 文件。"""
        q_lst, a_lst, history_lst = [], [], []
        for per in data:
            history, q, a = per['history'], per['q'], per['a']

            # 数据筛选条件
            if (contain_history and not history) or not q or not a:
                continue
            if len(q) < 10 or len(a) < 5:
                continue
            if len(q) > 256 or len(a) > 256:
                continue
            if not (chinese_ratio(q) > 0.9 and chinese_ratio(a) > 0.9):
                continue

            # 将有效数据添加到列表
            q_lst.append(q)
            a_lst.append(a)
            if contain_history:
                history_lst.append(history)
            else:
                history_lst.append([])

        # 创建 DataFrame 并写入 CSV 文件
        df = pd.DataFrame({'history': history_lst, 'q': q_lst, 'a': a_lst})
        df.to_csv(f'./dataset/{file_name}', mode='a', header=False, index=False, 
                   lineterminator='\r\n', escapechar='\\', quoting=csv.QUOTE_MINIMAL)

    chunk_size = 1000  # 每次处理的记录数
    data = []

    # 创建 CSV 文件并写入表头
    with open(f'./dataset/{file_name}', 'w', encoding='utf-8') as f:
        f.write('history,q,a\n')

    sft_datasets = ['./dataset/sft_data_zh.jsonl']
    
    # 开始处理数据集
    for path in sft_datasets:
        with jsonlines.open(path) as reader:
            total_lines = sum(1 for _ in open(path))  # 获取总行数
            
            # 使用 tqdm 创建进度条
            with tqdm(total=total_lines, desc="Processing lines") as pbar:
                for idx, obj in enumerate(reader):
                    try:
                        data.append({
                            'history': obj.get('history', ''),
                            'q': obj.get('input', '') + obj.get('q', ''),
                            'a': obj.get('output', '') + obj.get('a', '')
                        })

                        # 每达到 chunk_size，就处理并写入数据
                        if len(data) >= chunk_size:
                            process_and_write_data(data)
                            data = []
                            pbar.update(chunk_size)  # 更新进度条

                    except jsonlines.InvalidLineError as e:
                        print(f"Skipping invalid JSON line {idx + 1}: {e}")
                        continue

                # 处理剩余的数据
                if data:
                    process_and_write_data(data)
                    pbar.update(len(data))  # 更新进度条

    print("数据处理完成！")

In [16]:
sft_process(contain_history=False)

Processing lines: 100%|██████████| 11381621/11381621 [02:59<00:00, 63243.30it/s]

数据处理完成！


- 进行指令微调

In [257]:
# !deepspeed --master_port 29500 --num_gpus=2 full_sft.py --out_dir out --epochs 5

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/9f1b2321dc79550b1e0f194e165ff22.png" alt="9f1b2321dc79550b1e0f194e165ff22" style="zoom:50%;" />

- 对话测试

In [1]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer
from model.model import Transformer
from model.LMConfig import LMConfig

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. 设置设备和随机种子
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

setup_seed(1337)

In [3]:
# 2. 初始化模型和分词器
lm_config = LMConfig()
max_seq_len = 1024  # 可以根据需要调整
lm_config.max_seq_len = max_seq_len

model = Transformer(lm_config).to(device)
model_path = './out/full_sft_512.pth'  # 替换为你的模型路径
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()

Transformer(
  (tok_embeddings): Embedding(6400, 512)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-7): 8 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=512, out_features=512, bias=False)
        (wk): Linear(in_features=512, out_features=256, bias=False)
        (wv): Linear(in_features=512, out_features=256, bias=False)
        (wo): Linear(in_features=512, out_features=512, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
      (feed_forward): FeedForward(
        (w1): Linear(in_features=512, out_features=1408, bias=False)
        (w2): Linear(in_features=1408, out_features=512, bias=False)
        (w3): Linear(in_features=512, out_features=1408, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=512, 

In [4]:
tokenizer = AutoTokenizer.from_pretrained('MateConv/model/mateconv_tokenizer') 

In [5]:
# 3. 对话函数：生成完整回复
def generate_reply(prompt, temperature=0.5, top_k=16, stream=True):
    messages = [{"role": "user", "content": prompt}]
    
    # 使用自定义的 prompt 模板 (根据你的应用逻辑)
    new_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )[-(max_seq_len - 1):]

    input_ids = tokenizer(new_prompt).data['input_ids']
    input_ids = torch.tensor(input_ids, dtype=torch.long, device=device).unsqueeze(0)

    generated_text = ""
    with torch.no_grad():
        # 生成器返回的生成结果
        res_y = model.generate(input_ids, 
                               tokenizer.eos_token_id, 
                               max_new_tokens=max_seq_len, 
                               temperature=temperature, 
                               top_k=top_k, 
                               stream=stream)

        # 从生成器逐步获取生成结果
        try:
            y = next(res_y)
        except StopIteration:
            print("No answer")
            return ""

        history_idx = 0
        while y is not None:
            answer = tokenizer.decode(y[0].tolist())
            if answer and answer[-1] == '�':
                try:
                    y = next(res_y)
                except StopIteration:
                    break
                continue

            if len(answer):
                generated_text += answer[history_idx:]
            
            try:
                y = next(res_y)
            except StopIteration:
                break
            history_idx = len(answer)

    return generated_text

In [9]:
response = generate_reply("你好,好久不见！")
response

'你好，很高兴认识你！请问你有什么需要帮助的地方吗？'

In [10]:
response = generate_reply("你知道光速是多少吗？")
response

'光速是指光在真空中传播的速率，通常用符号c表示。根据国际单位制（SI）的定义，光速的常数为299,792,458米/秒。这个事实是物理学中非常重要的一个定理，它告诉我们在不同介质中，光速的大小是不同的。光速在真空中是不存在的，因为光速在真空中是不存在的。光速在真空中是不存在的，因为光在真空中只能传播，所以光速在真空中是不存在的。这个定律表明，任何传播光的速度都是相对于光速的，无论是从哪个方向传播，光速都是恒定的。光速在真空中是不存在的，因为光速不是一个固定的数值，而是不是一个固定的数值。因此，光速不存在。总之，光速是一个非常基本的物理常数，它在真空中是不存在的，但它是不存在的。'

In [15]:
response = generate_reply("世界上最好的大学是哪所大学？")
response

'世界上最好的大学是位于意大利博洛尼亚市的博洛尼亚大学（University，of，Bologna）该大学成立于1096年，是意大利最古老、最著名的高等教育机构之一。它位于意大利博洛尼亚市的中央，是意大利最古老、最著名的大学之一。博洛尼亚大学在欧洲和亚洲都有很强的声誉。博洛尼亚大学的历史可以追溯到11世纪，当时该大学在意大利建立，成为意大利最古老、最著名的大学之一。在那时，教会成员们主要从事教学和研究，这使博洛尼亚大学成为欧洲最好的大学之一。在中世纪和文艺复兴时期，博洛尼亚大学的许多重要事件都发生了变化。例如，在19世纪初，该大学开始扩展其教育范围，并逐渐扩大其教学范围。在20世纪，博洛尼亚大学开始扩大其教育范围，包括更广泛的课程和更广泛的学科领域。在20世纪，博洛尼亚大学成为了一所重要的学术机构，并为该大学的发展做出了重要贡献。在20世纪，博洛尼亚大学继续在全球范围内开设了许多分支机构，包括博洛尼亚大学、圣彼得堡大学和圣安德鲁堡大学等。这些机构和组织为博洛尼亚大学的学术生涯和研究生教育做出了重要的贡献。总之，博洛尼亚大学是意大利最好的大学之一，它在欧洲和亚洲都有着深厚的声誉，并为该大学的声誉和影响力提供了重要的基础。'

In [16]:
response = generate_reply("大气的主要成分是什么？")
response

'大气的主要成分是氮气（Nitrogen，N2）和氧气（Oxygen，O2）'

In [17]:
response = generate_reply("请问什么是机器学习？")
response

'机器学习是一种人工智能领域的技术，它利用计算机算法和统计学方法，让计算机系统能够自动地从数据中学习，并根据学习到的知识作出预测或决策。机器学习是通过使用统计学和数学方法，让计算机系统从数据中学习，并根据学习到的知识来进行预测或决策。机器学习可以分为三类：监督学习、无监督学习和强化学习。在监督学习中，计算机系统会根据已有的标记数据进行训练，以预测新数据的标记。例如，利用监督学习算法，计算机可以学习如何将输入数据映射到已知的输出数据。在无监督学习中，计算机系统会从未标记的数据中学习，并发现其中的模式和结构。例如，聚类算法可以将一组数据分成不同的群组，以便更好地理解数据。在强化学习中，计算机系统通过与环境的交互来学习如何做出正确的决策。例如，计算机系统可以通过与环境进行交互来学习如何做出最佳的决策。机器学习在许多领域都有广泛的应用，包括自然语言处理、图像和语音识别、金融预测、医学诊断和自动驾驶汽车等。它已经成为许多行业的重要技术，并在未来继续成为人工智能领域的重要组成部分。'

In [18]:
response = generate_reply("请问机器学习和深度学习的区别是什么？")
response

'机器学习和深度学习都是人工智能领域中的子领域，它们有一些共同的特点，但也有一些不同之处。机器学习是一种通过算法和模型来让计算机从数据中学习规律和模式的方法。机器学习算法可以从数据中学习，并根据这些学习来进行预测或决策。机器学习算法可以分为监督学习、无监督学习和强化学习。监督学习是最常见的机器学习类型，它需要给算法一个带标签的数据集，让它学习如何预测新数据的标签。无监督学习则不需要标签，它需要从数据中自动学习模式和结构。强化学习则是通过试错来学习，它需要根据环境的反馈来调整策略，以获得最大的回报。深度学习是机器学习的一种，它使用深度神经网络来模拟人类大脑的工作方式。深度学习算法可以从大量数据中学习，并自动发现数据中的模式和规律，从而实现更准确的预测和决策。深度学习在计算机视觉、自然语言处理、语音识别等领域中取得了巨大的成功。深度学习的算法可以自动地从数据中提取特征，并自动地进行分类、识别、预测等任务。深度学习算法通常需要大量的数据和计算资源，但它们可以处理大量的数据，并且可以自动地从数据中提取特征。深度学习算法可以自动地从数据中提取特征，并自动地进行分类、识别、回归等任务。因此，机器学习和深度学习都是人工智能领域中的重要分支，它们在不同的应用场景中都有着重要的作用。'

In [23]:
response = generate_reply("帮我建立一个逻辑回归模型？")
response

'要建立一个逻辑回归模型，需要进行以下步骤：1.数据预处理：将数据集分为训练集和测试集，通常采用70%的数据作为训练集，30%的数据作为测试集。2.特征工程：将原始的特征转换成可以被模型理解的形式。可以使用一些技术，如特征选择、特征提取、特征变换等，来提高模型的准确性。3.模型选择：选择一个适当的模型，如逻辑回归、支持向量机、决策树等。4.模型训练：使用训练集对模型进行训练，并使用交叉验证等技术来优化模型的参数和超参数。5.模型评估：使用测试集对模型进行评估，计算模型的准确率、精确率、召回率、F1分数等指标。6.模型优化：根据模型评估结果对模型进行优化，例如调整超参数、增加特征、调整模型超参数等。7.模型部署：将模型部署到生产环境中，并使用该模型进行预测、分类、聚类等任务。需要注意的是，建立逻辑回归模型需要一定的技术和经验，并且在实际应用中，需要根据实际情况进行调整和优化。'

- 测试openai API连接

In [19]:
from openai import OpenAI

In [13]:
client = OpenAI(api_key="key", 
                base_url="http://localhost:8000/v1")

In [14]:
completion = client.chat.completions.create(
  model="MateConv",
  messages=[
    {"role": "user", "content": "你好，好久不见！"}
  ]
)

In [15]:
completion

ChatCompletion(id='5ce4080e66844cca8f9c796666ee95ff', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='您好！我很高兴能够为您解答问题。请问您有什么问题需要我帮忙解答吗？', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727595610, model='MateConv', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None)

In [16]:
completion.choices[0].message.content

'您好！我很高兴能够为您解答问题。请问您有什么问题需要我帮忙解答吗？'

In [17]:
completion = client.chat.completions.create(
  model="MateConv",
  messages=[
    {"role": "user", "content": "请问，什么是人工智能？"}
  ]
)
completion.choices[0].message.content

'人工智能是一种使计算机系统具有人类智能的技术，它通过学习、推理、自然语言处理等技术来实现自主推理和决策。通过机器学习、深度学习等技术，人工智能可以处理大量的数据并进行自我学习，从而实现对问题的解决。'

In [18]:
completion = client.chat.completions.create(
  model="MateConv",
  messages=[
    {"role": "user", "content": "请问，什么是深度学习？"}
  ]
)
completion.choices[0].message.content

' 深度学习是一种机器学习的方法，它使用大量的数据来训练神经网络。深度学习通常使用多层神经网络来实现各种任务，例如图像识别、自然语言处理和语音识别。深度学习可以用于各种任务，如语音识别、图像识别、机器翻译和推荐系统等。'

In [19]:
completion = client.chat.completions.create(
  model="MateConv",
  messages=[
    {"role": "user", "content": "你好，好久不见！"}
  ]
)
completion.choices[0].message.content

'您好！我也很高兴见到您。有什么需要帮助的吗？'